<a href="https://colab.research.google.com/github/Karula-k/triplet_loss_speaker_recognition/blob/main/baseipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> buat alur

> model biasa

>~~model CNN~~

>~~model LSTM versi biasa~~

>~~save cart and tensorboard~~

> ~~ga buat cari best case~~

> besok pagi jam 8 kelar

>masalah model shape belum kelar

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import librosa
import soundfile as sf
import tensorflow as tf
import glob
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
path = "/content/drive/MyDrive/Colab Notebooks/self-data"

In [4]:
# #testing the image for refrence
# y, sr = librosa.load('/content/drive/MyDrive/Colab Notebooks/self-data/Sri mulyani/srimulyani-50.wav')
# stft_db = np.abs(librosa.stft(y))
# plt.figure(figsize=(14, 9))

# plt.figure(1)
# plt.subplot(211)
# plt.title('Audioform')
# librosa.display.waveshow(y, sr=sr)

# plt.subplot(212)
# plt.title('Spectrogram')
# librosa.display.specshow(librosa.amplitude_to_db(stft_db,ref=np.max), y_axis='log',sr=sr, x_axis='time')

In [5]:
class data_preparation:
  def __init__(self,dir,extractType,split_val):
    self.dir = dir
    self.extractType = extractType
    self.split_val = split_val
  def extract_features(self,audio_file):
    """Extract MFCC features from an audio file, shape=(TIME, MFCC)."""
    waveform, sample_rate = sf.read(audio_file)
     # Convert to mono-channel.
    if len(waveform.shape) == 2:
        waveform = librosa.to_mono(waveform.transpose())

    # Convert to 16kHz.
    if sample_rate != 16000:
        waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr= 16000)
    if self.extractType == "concanate":
      features_mfcc = librosa.feature.mfcc(y=waveform, sr=sample_rate, n_mfcc=40).transpose()
      features_LPC = librosa.core.lpc(waveform,order=39)
      return np.vstack((features_LPC,features_mfcc))
    else:
      features = librosa.feature.mfcc(y=waveform, sr=sample_rate, n_mfcc=40)
      return features.transpose()
  def get_dataset(self):
    """Get the dict from speaker to list of utterances for dataset."""
    wav_files = glob.glob(os.path.join(self.dir,"*/*.wav"))
    spk_to_utts = []
    labels = []
    for wav_file in wav_files:
        basename = os.path.basename(wav_file)
        split_name = basename.split("-")
        spk = split_name[0]
        spk_to_utts.append(self.extract_features(wav_file))
        labels.append(spk)
    return spk_to_utts,labels
  def prepare_targets(self,y):
    le = LabelEncoder()
    le.fit(y)
    y_train_enc = le.transform(y)
    return y_train_enc
  def split_data(self):
    x,y = self.get_dataset()
    y = self.prepare_targets(y)
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=self.split_val)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=self.split_val)
    del x, y
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    X_validation = np.array(X_validation)
    return X_train, X_test,X_validation, y_train, y_validation, y_test

In [30]:
# Function to create the forward model
def create_forward_lstm_model(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=input_shape))
    return model

# Function to create the backward model
def create_backward_lstm_model(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=input_shape, go_backwards=True))
    return model

# Combine the forward and backward models with fully connected layers and L2 normalization
def create_combined_model(input_shape):
    """https://sci-hub.ee/https://ieeexplore.ieee.org/abstract/document/7953194"""
    forward_model = create_forward_lstm_model(input_shape)
    backward_model = create_backward_lstm_model(input_shape)

    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Get the output from the forward and backward models
    forward_output = forward_model(input_layer)
    backward_output = backward_model(input_layer)
    concatenated_output =  tf.keras.layers.concatenate([forward_output, backward_output],axis=1)
    # Add fully connected layers
    x = tf.keras.layers.Dense(128, activation='relu')(concatenated_output)
    x = tf.keras.layers.Dense(64, activation='relu')(x)

    # Add L2 normalization
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.Dense(5, activation='softmax')(x)

    # Create the combined model
    combined_model = tf.keras.models.Model(inputs=input_layer, outputs=x)

    return combined_model

In [7]:
class OneD_CNN_Triplet(tf.keras.Model):
    """https://github.com/iPRoBe-lab/1D-Triplet-CNN/blob/master/models/OneD_Triplet_CNN.py"""
    def __init__(self, num_classes, input_shape=(313, 40, 1)):
        super().__init__()
        self.features = tf.keras.Sequential([
            tf.keras.layers.Input(shape=input_shape),
            tf.keras.layers.ZeroPadding2D(padding=((0, 0), (2, 2))),
            tf.keras.layers.Conv2D(16, (3, 1), strides=1, padding="valid", dilation_rate=(2, 1), activation='selu'),
            tf.keras.layers.Conv2D(32, (3, 1), strides=1, padding="valid", dilation_rate=(2, 1), activation='selu'),
            tf.keras.layers.Conv2D(64, (7, 1), padding="valid", dilation_rate=(2, 1), activation='selu'),
            tf.keras.layers.ZeroPadding2D(padding=((0, 0), (0, 1))),
            tf.keras.layers.Conv2D(128, (9, 1), strides=1, dilation_rate=(3, 1), activation='selu'),
        ])
        self.regularization = tf.keras.Sequential([
            tf.keras.layers.AlphaDropout(0.25)
        ])

        self.fc = tf.keras.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

    def call(self, x):
        o = self.features(x)
        o = self.regularization(o)
        o = self.fc(o)
        return o

In [56]:
def model_type(model):
  if model =='LSTM':
    return tf.keras.Sequential([
    tf.keras.layers.LSTM(64,return_sequences=True, input_shape=(None, 40)),
    tf.keras.layers.LSTM(64,return_sequences=True,activation='relu'),
    tf.keras.layers.LSTM(64,activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax'),
])
  elif model =='OneD_CNN':
    model = OneD_CNN_Triplet(num_classes=5)
    model.build(input_shape=(None,313, 40, 1))
    return model
  elif model == 'TRISTOUNET':
    input_dim = (None, 40)
    model = create_combined_model(input_dim)
    return model
  else:
      return tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None,40)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation='softmax'),
])

In [9]:
def main(extract_type,model_name,split,lr):
  lr = 1 * 10**(-lr)
  print('='*10,'data preperation')
  data = data_preparation(path,extract_type,split)
  X_train, X_test,X_validation, y_train, y_validation, y_test =  data.split_data()
  name = f'{extract_type} {model_name}'
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=f'./Graph/{name}', histogram_freq=0,
        write_graph=True, write_images=True)
  print('='*10,'training')
  model = model_type(model_name)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])
  history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=50,callbacks=[tb_callback])
  test_loss, test_accuracy = model.evaluate(X_test, y_test)
  return history.history['accuracy'], test_loss, test_accuracy

In [10]:
data = data_preparation(path,'MFCC',0.2)

In [11]:
X_train, X_test,X_validation, y_train, y_validation, y_test =  data.split_data()

In [12]:
X_train.shape

(284, 313, 40)

In [13]:
y_train.shape

(284,)

In [60]:
model = model_type('LSTM')
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, None, 64)          26880     
                                                                 
 lstm_11 (LSTM)              (None, None, 64)          33024     
                                                                 
 lstm_12 (LSTM)              (None, 64)                33024     
                                                                 
 dense_60 (Dense)            (None, 5)                 325       
                                                                 
Total params: 93253 (364.27 KB)
Trainable params: 93253 (364.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=50)
test_loss, test_accuracy = model.evaluate(X_test, y_test)

Epoch 1/50
9/9 [==============================] - 71s 857ms/step - loss: 1.5510 - accuracy: 0.3873 - val_loss: 138.5885 - val_accuracy: 0.4722
Epoch 2/50
9/9 [==============================] - 4s 423ms/step - loss: 4.2614 - accuracy: 0.5915 - val_loss: 1.3519 - val_accuracy: 0.5694
Epoch 3/50
9/9 [==============================] - 4s 422ms/step - loss: 1.3135 - accuracy: 0.6655 - val_loss: 1.3541 - val_accuracy: 0.6111
Epoch 4/50
9/9 [==============================] - 5s 587ms/step - loss: 7.8266 - accuracy: 0.6549 - val_loss: 26.3302 - val_accuracy: 0.4306
Epoch 5/50
9/9 [==============================] - 5s 592ms/step - loss: nan - accuracy: 0.2535 - val_loss: nan - val_accuracy: 0.1806
Epoch 6/50
9/9 [==============================] - 4s 416ms/step - loss: nan - accuracy: 0.2042 - val_loss: nan - val_accuracy: 0.1806
Epoch 7/50
9/9 [==============================] - 4s 422ms/step - loss: nan - accuracy: 0.2042 - val_loss: nan - val_accuracy: 0.1806
Epoch 8/50
9/9 [==================

In [62]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 98ms/step - loss: nan - accuracy: 0.2000


In [ ]:
a = main('dense','MFCC',0.8,3)

========== data preperation
========== training
Epoch 1/50


InvalidArgumentError: ignored

In [ ]:
def fitness_function(hyperparameter):
    hyperparameter = encode(hyperparameter)
    print(hyperparameter)
    accuracy = main(*hyperparameter)
    return accuracy

In [ ]:
def mutation(domain, schedule, prob):
  gene = random.randint(0, len(domain) - 1)
  #print(gene)
  mutant = schedule
  if random.random() < prob: # 0.02, 0.05
    if schedule[gene] != domain[gene][0]:
      mutant = schedule[0:gene] + [schedule[gene] - 1] + schedule[gene + 1:]
    else:
      if schedule[gene] != domain[gene][1]:
        mutant = schedule[0:gene] + [schedule[gene] + 1] + schedule[gene + 1:]
  return mutant

In [ ]:
def crossover(domain, individual1, individual2):
  gene = random.randint(1, len(domain) - 2)
  # print(gene)
  return individual1[0:gene] + individual2[gene:]

> perbaiki load dan save ambil 20 terakhir pake pandas

> besok harus nulis

> kasih nama row

In [ ]:
import pandas as pd
import csv
import random

In [ ]:
# def load_population_from_csv(csv_filename):
#     try:
#         with open(csv_filename, 'r') as file:
#             reader = csv.reader(file)
#             header = next(reader)
#             current_generation = int(header[0])
#             population = [list(map(int, row)) for row in reader]
#         return current_generation, population
#     except FileNotFoundError:
#         return None, None, None

In [ ]:
def load_population_from_csv(csv_filename):
  if os.path.exists(csv_filename):
    reader = pd.read_csv(csv_filename)
    last_twenty = reader.tail(20)
    generation = last_twenty['Generation'].values[-1]
    genes_columns = ['Gene_0', 'Gene_1', 'Gene_2', 'Gene_3']
    population = last_twenty[genes_columns].values.tolist()
    return generation,population
  else:
    return 0,None

In [ ]:
def save_population_to_csv(csv_filename, current_generation, populations,resume):
    with open(csv_filename, 'a+', newline='') as file:
        writer = csv.writer(file)
        if not resume:
          writer.writerow(["Generation",] + [f"Gene_{i}" for i in range(len(populations[0]))])
        for population in populations:
          row = [current_generation,] + population
          writer.writerow(row)

In [ ]:
# save_population_to_csv('/content/drive/MyDrive/Colab Notebooks/results_csv/x_test_base.csv',1,[[1,2,3,4],[1,2,3,4]],True)

In [ ]:
def genetic_algorithm(domain, fitness_function, csv_filename,resume, population_size=20, elitism=0.2,
                      number_of_generations=5, mutation_probability=0.05):
    current_generation,population = load_population_from_csv(csv_filename)
    if population is None:
        population = []
        for i in range(population_size):
            individual = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
            population.append(individual)

    number_elitism = int(elitism * population_size)
    # for i in range(number_of_generations):
    while current_generation < number_of_generations:

        costs = [(fitness_function(individual), individual) for individual in population]

        costs.sort(reverse=True)
        ordered_individuals = [individual for (cost, individual) in costs]
        save_population_to_csv(csv_filename,current_generation,ordered_individuals,resume)
        population = ordered_individuals[0:number_elitism]
        while len(population) < population_size:
            i1 = random.randint(0, number_elitism)
            i2 = random.randint(0, number_elitism)
            new_individual = crossover(domain, ordered_individuals[i1], ordered_individuals[i2])
            new_individual_mutation = mutation(domain, new_individual, mutation_probability)
            population.append(new_individual_mutation)
        current_generation+=1
        resume  = True
    return costs[0][1]

In [ ]:
def encode(gene):
  first_gene = gene[:3]
  second_gene = gene[3:]
  model_types = {0:"LSTM",1:"dense",2:"OneD_CNN",3:"TRISTOUNET"}
  extract_types = {0:"MFCC",1:"concatenate"}
  splits = {0:0.3,1:0.2,2:0.1}
  index_dict = [model_types,extract_types,splits]
  return [dict_place[index] for index,dict_place in zip(first_gene,index_dict)] + second_gene

In [ ]:
def decode(gene):
  first_gene = gene[:3]
  second_gene = gene[3:]
  model_types = {"LSTM":0,"dense":1,"OneD_CNN":2,"TRISTOUNET":3}
  extract_types = {"MFCC":0,"concatenate":1}
  splits = {0.3:0,0.2:1,0.1:2}
  index_dict = [model_types,extract_types,splits]
  return [dict_place[index] for index,dict_place in zip(first_gene,index_dict)]+ second_gene

In [ ]:
domain = [(0, 3), (0, 1),(0, 2),(1, 6)]
result = genetic_algorithm(domain, fitness_function, '/content/drive/MyDrive/Colab Notebooks/results_csv/test_base.csv',False)
print("Best solution:", result)

['dense', 'concatenate', 0.3, 3]
Epoch 1/50


InvalidArgumentError: ignored

In [ ]:
# 'https://openaccess.thecvf.com/content/CVPR2022W/WAD/papers/Marinello_TripletTrack_3D_Object_Tracking_Using_Triplet_Embeddings_and_LSTM_CVPRW_2022_paper.pdf'

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=Graph

<IPython.core.display.Javascript object>